In [0]:
from datetime import datetime, timedelta
import requests
import os
from pyspark.sql import SparkSession

In [0]:
# Cria sessão Spark
spark = SparkSession.builder.getOrCreate()

In [0]:
# =====================================
# PARÂMETROS DO JOB (Widgets)
# =====================================
dbutils.widgets.text("data_ref", "", "Data de Referência (AAAAMMDD)")
dbutils.widgets.text("url_base", "", "URL base (opcional)")
dbutils.widgets.text("destino", "", "Destino no Volume")

In [0]:
# Lendo parâmetros
data_ref = dbutils.widgets.get("data_ref")
url_base = dbutils.widgets.get("url_base")
destino_base = dbutils.widgets.get("destino")

# =====================================
# VALIDAÇÃO E DEFAULTS
# =====================================

In [0]:
# 1) Se não informar data, usa hoje
if not data_ref:
    data_ref = datetime.now().strftime("%Y%m%d")

# 2) URL base
if not url_base:
    url_base = "https://dataserver-coids.inpe.br/queimadas/queimadas/riscofogo_meteorologia/observado/risco_fogo"

url_base = url_base.rstrip("/")

# 3) Destino base
if not destino_base:
    destino_base = "/Volumes/datamasters/raw/raw_inpe"

# Garante que destino existe
os.makedirs(destino_base, exist_ok=True)

# =====================================
# INPUTS
# =====================================

In [0]:
print("=====================================")
print(f"Data de referência : {data_ref}")
print(f"URL de origem      : {url_base}")
print(f"Destino final      : {destino_base}")
print("=====================================")

# =====================================
#  FUNÇÃO PARA BAIXAR ÚLTIMO ARQUIVO DISPONÍVEL (COM CACHE)
#  =====================================

In [0]:
def baixar_ultimo_disponivel(data_referencia: str, destino_dir: str, url_base: str):
    data = datetime.strptime(data_referencia, "%Y%m%d")

    while True:
        data_str = data.strftime("%Y%m%d")
        file_name = f"INPE_FireRiskModel_2.2_FireRisk_{data_str}.nc"
        destino_final = os.path.join(destino_dir, file_name)
        url_nc = f"{url_base}/{file_name}"

        print(f"Tentando arquivo: {file_name}")

        # ============================================================
        # 1) CACHE — não baixa se já existir e for > 0 KB
        # ============================================================
        if os.path.exists(destino_final):
            file_size = os.path.getsize(destino_final)
            if file_size > 0:
                print(f"✔ Cache encontrado: {destino_final} ({round(file_size/1024,2)} KB)")
                return destino_final, data_str, url_nc
            else:
                print(f"⚠ Arquivo em cache está vazio (0 KB). Removendo e baixando novamente.")
                os.remove(destino_final)

        # ============================================================
        # 2) Baixa arquivo do INPE
        # ============================================================
        resp = requests.get(url_nc)

        if resp.status_code == 200:
            # Salva arquivo
            with open(destino_final, "wb") as f:
                f.write(resp.content)

            # ============================================================
            # 3) Valida tamanho do arquivo (> 0 KB)
            # ============================================================
            file_size = os.path.getsize(destino_final)

            if file_size == 0:
                print(f"⚠ Arquivo baixado está vazio. Removendo e tentando dia anterior...")
                os.remove(destino_final)
                data -= timedelta(days=1)
                continue

            print(f"Arquivo válido encontrado e salvo: {destino_final} ({round(file_size/1024,2)} KB) ✅")
            return destino_final, data_str, url_nc

        elif resp.status_code == 404:
            print(f"Arquivo {file_name} não encontrado (404). Tentando dia anterior...")
            data -= timedelta(days=1)
            continue

        else:
            raise RuntimeError(
                f"Erro inesperado ao acessar {url_nc} - HTTP {resp.status_code}"
            )

# =====================================
# EXECUÇÃO DO MÉTODO
# =====================================

In [0]:
destino_final, data_usada, url_usada = baixar_ultimo_disponivel(
    data_ref, destino_base, url_base
)

# =====================================
# LOG FINAL
# =====================================

In [0]:
print("=====================================")
print(f"Data solicitada    : {data_ref}")
print(f"Data utilizada     : {data_usada}")
print(f"URL utilizada      : {url_usada}")
print(f"Arquivo salvo em   : {destino_final}")
print("=====================================")

# =====================================
# LEITURA PARA VERIFICAÇÃO
# =====================================

In [0]:
file_size_kb = round(os.path.getsize(destino_final) / 1024, 2)
print(f"Tamanho final do arquivo: {file_size_kb} KB")

print("Observação: arquivos .nc (NetCDF) não são CSV — leitura deve ser feita com XArray ou rioxarray.")